* Budhdeo Kumar
* Information Technology Department
* Birla Institute of Technology , Patna
* budhdeo.abc@gmail.com
* JAN 31 , 2019

In [ ]:
# Import environment tools
import re
import itertools
import warnings
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import pandas as pd
import numpy as np
import keras

# Import plotly tools
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls

# Import keras tools
from keras import regularizers
from keras.callbacks import History 
from keras.layers import Dense, Input, Dropout
from keras.models import Sequential
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier

# Import other tools
from __future__ import print_function
from pandas import read_excel
from IPython.display import Image
from collections import Counter
from itertools import cycle
from scipy import stats, integrate, interp
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
import sklearn
# Gradient Boosters
import xgboost as xgb # Accuracy
import lightgbm as lgb # Speed

from sklearn import decomposition, preprocessing, svm
# Dimensionality Reduction
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
# Ensemble
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier, ExtraTreesClassifier
# Guassian
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
# Regression
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
# Bayesian
from sklearn.naive_bayes import GaussianNB
# Instance Based
from sklearn.neighbors import KNeighborsClassifier
# Nueral Network
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

# Import relevant machine learning analyis tools
from sklearn import metrics
#from sklearn.cross_validation import KFold, train_test_split
# Imputation
from sklearn.impute import SimpleImputer 
from sklearn.metrics import mean_absolute_error,roc_curve,accuracy_score,auc,roc_auc_score,confusion_matrix,precision_score,recall_score,f1_score, classification_report
from sklearn.metrics.cluster import fowlkes_mallows_score
from sklearn.model_selection import BaseCrossValidator, GridSearchCV, train_test_split,cross_val_score,cross_validate,cross_val_predict, KFold, StratifiedKFold, learning_curve
from sklearn.pipeline import Pipeline
# Standardization
from sklearn.preprocessing import LabelEncoder, StandardScaler, label_binarize


# Initial tool settings
%matplotlib inline
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
plt.style.use('fivethirtyeight')
sns.set(style='white', context='notebook', palette='deep')
py.init_notebook_mode(connected=True)

history = History()
random_state = 43

names = ["k-Nearest Neighbors",         
         "Support Vector Machine",
         "Linear SVM",
         "RBF SVM",
         "Gaussian Process",
         "Decision Tree",
         "Extra Trees",
         "Random Forest",
         "Extra Forest",
         "AdaBoost",
         "Gaussian Naive Bayes",
         "LDA",
         "QDA",
         "Logistic Regression",
         "SGD Classifier",
         "Multilayer Perceptron",
         "Voting Classifier"
        ]

algorithms = [ KNeighborsClassifier(n_neighbors=3),
               SVC(random_state=random_state),
               SVC(kernel="linear",random_state=random_state),
               SVC(kernel="rbf",random_state=random_state),
               GaussianProcessClassifier(),
               DecisionTreeClassifier(random_state=random_state),
               ExtraTreesClassifier(random_state=random_state),
               RandomForestClassifier(random_state=random_state),
               GradientBoostingClassifier(random_state=random_state),
               AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state),n_estimators=10,learning_rate=0.1,random_state=random_state),
               GaussianNB(),
               LinearDiscriminantAnalysis(),
               QuadraticDiscriminantAnalysis(),
               LogisticRegression(random_state=random_state),
               SGDClassifier(),               
               MLPClassifier(hidden_layer_sizes=(100,),momentum=0.9,solver='sgd',random_state=random_state),
               VotingClassifier(estimators=[('log', LogisticRegression()), ('SVM',SVC(C=1000)), ('MLP', MLPClassifier(hidden_layer_sizes=(100,)))], voting='hard')
              ]
#algorithms.append(SVC(random_state=random_state))

classifiers = {  "k-Nearest Neighbors" : KNeighborsClassifier(n_neighbors=3),
                 "Support Vector Machine" :  SVC(random_state=random_state),
                 "Linear SVM" :  SVC(kernel="linear",random_state=random_state),
                 "RBF SVM" :  SVC(kernel="rbf",random_state=random_state),
                 "Gaussian Process" : GaussianProcessClassifier(),
                 "Decision Tree" : DecisionTreeClassifier(random_state=random_state),
                 "Extra Trees" : ExtraTreesClassifier(random_state=random_state),
                 "Random Forest" : RandomForestClassifier(random_state=random_state),
                 "Extra Forest" : GradientBoostingClassifier(random_state=random_state),
                 "AdaBoost" : AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state),n_estimators=10,random_state=random_state,learning_rate=0.1),
                 "Gaussian Naive Bayes" : GaussianNB(),
                 "LDA" : LinearDiscriminantAnalysis(),
                 "QDA" :  QuadraticDiscriminantAnalysis(),
                 "Logistic Regression" : LogisticRegression(random_state=random_state),
                 "SGD Classifier" : SGDClassifier(),
                 "Multilayer Perceptron" :  MLPClassifier(hidden_layer_sizes=(100,),momentum=0.9,solver='sgd',random_state=random_state),
                 "Voting Classifier" : VotingClassifier(estimators=[('log', LogisticRegression()), ('SVM',SVC(C=1000)), ('MLP', MLPClassifier(hidden_layer_sizes=(100,)))], voting='hard')
              }
print("Done!")

**Import Data**

In [ ]:
df=pd.read_csv("../input/heart-disease-with-76-attributes/cleveland_data.csv",header=None)

**Data Cleaning**
* Milestones in data cleaning:
1. 1.  Information of each person is seperated in 3 or 4 or 5 rows.
1. 2.  Missing values
1. 3.  Untidy Dataset

**Conversion of datatype of every data for data cleaning**
* Change each data to string dtype 

In [ ]:
rw=[]
for row_index,row in df.iterrows():
    x=str(row)
    rw.append(list(map(str,x.split())))
row1=list(df[0])

A Python function to parse and extract the data of 76 columns for each row and select only those rows which have every data for 76 columns.

In [ ]:
row1=list(df[0])
row1[0]='1 0 63 1 9 9 9'
row2=[]
s=''
for i in row1:
    if(i[-1]=='e'):
        s=s+" "+i
        row2.append(s)
        s=''
    else:
        s=s+" "+i
for i in range(len(row2)):
    row2[i]=list(row2[i].split())
fault=[]
for i in range(len(row2)):
    if(len(row2[i]) != 76):
        fault.append(i)
row3=row2[:282]
data = pd.DataFrame(row3)

In [ ]:
data

**76 features in dataset**
* Complete attribute documentation: 
* 1 id: patient identification number 
* 2 ccf: social security number (I replaced this with a dummy value of 0) 
* 3 age: age in years 
* 4 sex: sex (1 = male; 0 = female) 
* 5 painloc: chest pain location (1 = substernal; 0 = otherwise) 
* 6 painexer (1 = provoked by exertion; 0 = otherwise) 
* 7 relrest (1 = relieved after rest; 0 = otherwise) 
* 8 pncaden (sum of 5, 6, and 7) 
* 9 cp: chest pain type 
* -- Value 1: typical angina 
* -- Value 2: atypical angina 
* -- Value 3: non-anginal pain 
* -- Value 4: asymptomatic 
* 10 trestbps: resting blood pressure (in mm Hg on admission to the hospital) 
* 11 htn 
* 12 chol: serum cholestoral in mg/dl 
* 13 smoke: I believe this is 1 = yes; 0 = no (is or is not a smoker) 
* 14 cigs (cigarettes per day) 
* 15 years (number of years as a smoker) 
* 16 fbs: (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false) 
* 17 dm (1 = history of diabetes; 0 = no such history) 
* 18 famhist: family history of coronary artery disease (1 = yes; 0 = no) 
* 19 restecg: resting electrocardiographic results 
* -- Value 0: normal 
* -- Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV) 
* -- Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria 
* 20 ekgmo (month of exercise ECG reading) 
* 21 ekgday(day of exercise ECG reading) 
* 22 ekgyr (year of exercise ECG reading) 
* 23 dig (digitalis used furing exercise ECG: 1 = yes; 0 = no) 
* 24 prop (Beta blocker used during exercise ECG: 1 = yes; 0 = no) 
* 25 nitr (nitrates used during exercise ECG: 1 = yes; 0 = no) 
* 26 pro (calcium channel blocker used during exercise ECG: 1 = yes; 0 = no) 
* 27 diuretic (diuretic used used during exercise ECG: 1 = yes; 0 = no) 
* 28 proto: exercise protocol 
* 1 = Bruce 
* 2 = Kottus 
* 3 = McHenry 
* 4 = fast Balke 
* 5 = Balke 
* 6 = Noughton 
* 7 = bike 150 kpa min/min (Not sure if "kpa min/min" is what was written!) 
* 8 = bike 125 kpa min/min 
* 9 = bike 100 kpa min/min 
* 10 = bike 75 kpa min/min 
* 11 = bike 50 kpa min/min 
* 12 = arm ergometer 
* 29 thaldur: duration of exercise test in minutes 
* 30 thaltime: time when ST measure depression was noted 
* 31 met: mets achieved 
* 32 thalach: maximum heart rate achieved 
* 33 thalrest: resting heart rate 
* 34 tpeakbps: peak exercise blood pressure (first of 2 parts) 
* 35 tpeakbpd: peak exercise blood pressure (second of 2 parts) 
* 36 dummy 
* 37 trestbpd: resting blood pressure 
* 38 exang: exercise induced angina (1 = yes; 0 = no) 
* 39 xhypo: (1 = yes; 0 = no) 
* 40 oldpeak = ST depression induced by exercise relative to rest 
* 41 slope: the slope of the peak exercise ST segment 
* -- Value 1: upsloping 
* -- Value 2: flat 
* -- Value 3: downsloping 
* 42 rldv5: height at rest 
* 43 rldv5e: height at peak exercise 
* 44 ca: number of major vessels (0-3) colored by flourosopy 
* 45 restckm: irrelevant 
* 46 exerckm: irrelevant 
* 47 restef: rest raidonuclid (sp?) ejection fraction 
* 48 restwm: rest wall (sp?) motion abnormality 
* 0 = none 
* 1 = mild or moderate 
* 2 = moderate or severe 
* 3 = akinesis or dyskmem (sp?) 
* 49 exeref: exercise radinalid (sp?) ejection fraction 
* 50 exerwm: exercise wall (sp?) motion 
* 51 thal: 3 = normal; 6 = fixed defect; 7 = reversable defect 
* 52 thalsev: not used 
* 53 thalpul: not used 
* 54 earlobe: not used 
* 55 cmo: month of cardiac cath (sp?) (perhaps "call") 
* 56 cday: day of cardiac cath (sp?) 
* 57 cyr: year of cardiac cath (sp?) 
* 58 num: diagnosis of heart disease (angiographic disease status) 
* -- Value 0: < 50% diameter narrowing 
* -- Value 1: > 50% diameter narrowing 
* (in any major vessel: attributes 59 through 68 are vessels) 
* 59 lmt 
* 60 ladprox 
* 61 laddist 
* 62 diag 
* 63 cxmain 
* 64 ramus 
* 65 om1 
* 66 om2 
* 67 rcaprox 
* 68 rcadist 
* 69 lvx1: not used 
* 70 lvx2: not used 
* 71 lvx3: not used 
* 72 lvx4: not used 
* 73 lvf: not used 
* 74 cathef: not used 
* 75 junk: not used 
* 76 name: last name of patient (I replaced this with the dummy string "name")



In [ ]:
features=['id', 'ccf', 'age', 'sex', 'painloc', 'painexer', 'relrest', 'pncaden', 'cp', 'trestbps', 'htn', 'chol','smoke', 'cigs', 'years', 'fbs', 'dm', 'famhist', 'restecg', 'ekgmo', 'ekgday', 'ekgyr', 'dig', 'prop', 'nitr', 'pro', 'diuretic', 'proto', 'thaldur', 'thaltime', 'met', 'thalach', 'thalrest', 'tpeakbps', 'tpeakbpd', 'dummy', 'trestbpd', 'exang', 'xhypo', 'oldpeak', 'slope', 'rldv5', 'rldv5e', 'ca', 'restckm', 'exerckm', 'restef', 'restwm', 'exeref', 'exerwm', 'thal', 'thalsev', 'thalpul', 'earlobe', 'cmo', 'cday', 'cyr', 'pred_attribute', 'lmt', 'ladprox', 'laddist', 'diag', 'cxmain', 'ramus', 'om1', 'om2', 'rcaprox', 'rcadist', 'lvx1', 'lvx2', 'lvx3', 'lvx4', 'lvf', 'cathef', 'junk', 'name']

Setting up dataset with features as column name.

In [ ]:
data.columns=features

In [ ]:
data.head()

**Data Visualization**

In [ ]:
sns.countplot(x=data['dm'])
plt.show()

This plot shows that there is missing data in history of diabities in their family "dm" column which is non-medical attribute.

In [ ]:
r=list(data['cigs'])
l=list(data['smoke'])
for i in range(len(r)):
    if(int(r[i])>0):
        l[i]=1
    else:
        l[i]=0

These attributes are used for further model.

In [ ]:
attributes_used=['age','sex','cp','thalach','fbs','famhist','thalrest','trestbpd','exang','pred_attribute']

In [ ]:
m=[]
for i in range(len(features)):
    if(features[i] in attributes_used):
        pass
    else:
        m.append(i)

Drop the unused columns.

In [ ]:
data.drop(data.columns[m], axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
data = data.convert_objects(convert_numeric=True)

In [ ]:
data.dtypes

In [ ]:
data["pred_attribute"].replace(inplace=True, value=[1, 1, 1, 1], to_replace=[1, 2, 3, 4])

In [ ]:
data.sort_values('pred_attribute').head()

In [ ]:
data.head()

Distribution of data in each column of attributes used.

In [ ]:
plt.subplots(figsize=(20,15))
length=len(attributes_used)
for i,j in itertools.zip_longest(attributes_used,range(length)):
    plt.subplot((length/2),3,j+1)
    plt.subplots_adjust(wspace=0.2,hspace=0.5)
    data[i].hist(bins=20,edgecolor='black')
    plt.title(i)
plt.show()

In [ ]:
cont_attributes=['age','thalach','thalrest','trestbpd','pred_attribute']
disc_attributes=['cp','fbs','famhist','exang','pred_attribute']

Relation between attributes and prediction attributes.

In [ ]:
sns.pairplot(data=data[cont_attributes],hue='pred_attribute',diag_kind='kde')
#plt.gcf().set_size_inches(20,15)
plt.show()

In [ ]:
sns.pairplot(data=data[disc_attributes],hue='pred_attribute',diag_kind='kde')
#plt.gcf().set_size_inches(20,15)
plt.show()

Checking of outliers.

In [ ]:
sns.boxplot(data=data[cont_attributes])
#plt.gcf().set_size_inches(20,15)
plt.show()

In [ ]:
data[cont_attributes].describe()

In [ ]:
sns.countplot(x='pred_attribute',data=data)
plt.show()

In [ ]:
sns.countplot(x='sex',data=data)
plt.show()

In [ ]:
sns.countplot(x='fbs',data=data)
plt.show()

In [ ]:
#sns.countplot(x='smoke',data=data)
#plt.show()

In [ ]:
sns.countplot(x='cp',data=data)
plt.show()

In [ ]:
sns.countplot(x='famhist',data=data)
plt.show()

Check for null value.

In [ ]:
data.isnull().sum()

Setting up for Machine learnig model.

In [ ]:
#dataset = dataset.convert_objects(convert_numeric=True)

# Load data
X = data.iloc[:, :-1].values  
y = data.iloc[:, -1].values # = dataset.iloc[:, 12].values

#dataset.dropna(inplace=True, axis=0, how="any")
#X=dataset.loc[:, "age":"thal" ]
#y=dataset["pred_attribute"]
#np_X = np_dataset[:, :-1]  
#np_y = np_dataset[:, -1]  

my_imputer = SimpleImputer()
my_imputer = my_imputer.fit(X[:,0:10])   
X[:, 0:10] = my_imputer.transform(X[:, 0:10])

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

#features_all = dataset[dataset.columns[:13]]
#features_standard = scaler.fit_transform(dataset[dataset.columns[:13]]) # Gaussian Standardisation
#X = pd.DataFrame(features_standard,columns=[feature13])
#X['pred_attribute'] = dataset['pred_attribute']
#outcome = X['pred_attribute']

Splitting of dataset in train and test .

In [ ]:
# evaluate the model by splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

freqs = pd.DataFrame({"Training dataset": y_train.sum(),
                      "Test dataset":y_test.sum(),
                      "Total": y.sum()},
                     index=["Healthy", "Sick"])
freqs[["Training dataset", "Test dataset", "Total"]]


In [ ]:
def err_score(X_train, X_test, y_train, y_test):
    model =  RandomForestRegressor(random_state=random_state)
    model.fit(X_train, y_train)
    
    # predict class labels for the train set
    pred_train = model.predict(X_train)
    # predict class labels for the test set
    pred_test = model.predict(X_test)
    # check the mean absolute error on test set
    print("Mean Absolute Error from imputation: ", mean_absolute_error(y_test, pred_test))
    
#X = my_imputer.fit_transform(X)
#X_train = my_imputer.fit_transform(X_train)
#X_test = my_imputer.transform(X_test)
err_score(X_train, X_test, y_train, y_test)

In [ ]:
# instantiate a logistic regression model, and fit with X and y (with training data in X,y)
model = LogisticRegression(random_state = random_state)
model.fit(X_train, y_train)

# check the accuracy on the training set
print("Accuracy on training set: ", model.score(X_train, y_train))
# check the accuracy on the test set
print("Accuracy on test set: ", model.score(X_test, y_test))

In [ ]:
pred_train = model.predict(X_train)
#confusion_matrix(y_train,pred_train)
pd.crosstab(y_train, pred_train, rownames=['Predicted'], colnames=['Reality'], margins=True)


In [ ]:
pred_test = model.predict(X_test)
#confusion_matrix(y_test,pred_test)
pd.crosstab(y_test, pred_test, rownames=['Predicted'], colnames=['Reality'], margins=True)

In [ ]:
data1 = data[data.columns[:9]]
outcome = data['pred_attribute']

#train,test=train_test_split(dataset,test_size=0.25,random_state=0,stratify=dataset['pred_attribute'])# stratify the outcome
#train_X=train[train.columns[:13]]
#test_X=test[test.columns[:13]]
#train_Y=train['pred_attribute']
#test_Y=test['pred_attribute']

data_copy=[]
for model in algorithms:
    model.fit(X_train,y_train)
    pred_test = model.predict(X_test)
    data_copy.append(metrics.accuracy_score(pred_test, y_test))
    
models_df = pd.DataFrame(data_copy, index=names)   
models_df.columns=['Accuracy']
models_df

In [ ]:
sns.heatmap(data[data.columns[:10]].corr(),annot=True,cmap='RdYlGn')
fig=plt.gcf()
fig.set_size_inches(25,20)
#plt.figure(figsize=(14,12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
plt.show()

In [ ]:
#model = ExtraTreesClassifier(n_estimators=100,random_state=random_state)
#model.fit(X, y)
#pd.Series(model.feature_importances_, index=data1.columns).sort_values(ascending=False)

In [ ]:
model = RandomForestClassifier(n_estimators=100,random_state=random_state) # , max_features=(10 ** 0.5))
model.fit(X, y)
pd.Series(model.feature_importances_, index=data1.columns).sort_values(ascending=False)

In [ ]:
data_select = data[['age','sex','fbs','thalach','cp','exang','pred_attribute']]

X = data_select.iloc[:, :-1].values  
y = data_select.iloc[:, -1].values
#X = np_dataset[:, :-1]  
#y = np_dataset[:, -1]  

In [ ]:
my_imputer = my_imputer.fit(X[:,0:7])   
X[:, 0:7] = my_imputer.transform(X[:, 0:7])

In [ ]:
X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

In [ ]:
err_score(X_train, X_test, y_train, y_test)

In [ ]:
data_copy=[]
for model in algorithms:
    model.fit(X_train,y_train)
    pred_test = model.predict(X_test)
    data_copy.append(metrics.accuracy_score(pred_test, y_test))
    
models_df2 = pd.DataFrame(data_copy, index=names)  
models_df2.columns = ['New Accuracy']  

models_df2 = models_df2.merge(models_df, left_index=True, right_index=True, how='left')
models_df2['Increase'] = models_df2['New Accuracy'] - models_df2['Accuracy']
models_df2

In [ ]:
kfold = KFold(n_splits=10, random_state=random_state) # k=10, split the data into 10 equal parts
CV_results = []
# iterate over classifiers
Acc= {}
Acc_Train = {}
Acc_Test = {}
Std_Train={}
Std_Test={}
Predictions = {}
ROC = {}
AUC = {}
Gmean = {}
Precision = {}
Recall = {}
F1_score = {}
Confusion_Matrix = {}
mats = pd.DataFrame(Confusion_Matrix)

for clf in classifiers:
    #Acc[clf] = cross_validate(classifiers[clf],X=X,y=y,cv=10,n_jobs=-1,scoring='accuracy',return_train_score=True)
    Acc[clf] = cross_validate(classifiers[clf],X_train,y_train,cv=kfold,n_jobs=-1,scoring='accuracy',return_train_score=True)
    Acc_Train[clf] =  Acc[clf]['train_score'].mean()
    Acc_Test[clf] = Acc[clf]['test_score'].mean()
    Std_Train[clf] = Acc[clf]['train_score'].std()
    Std_Test[clf] = Acc[clf]['test_score'].std()
    CV_results.append(Acc[clf])
    
    classifiers[clf].fit(scaler.transform(X_train),y_train)
    pred =  classifiers[clf].predict(scaler.transform(X_test))
    ROC[clf] = roc_auc_score(y_test,pred)
    AUC[clf] = auc(y_test,pred,reorder=True)
    Gmean[clf] = fowlkes_mallows_score(y_test,pred)
    Precision[clf] = precision_score(y_test,pred)
    Recall[clf] = recall_score(y_test,pred)    
    F1_score[clf] = f1_score(y_test,pred)
    Confusion_Matrix[clf] = confusion_matrix(y_test,pred)

Accuracy_train = pd.DataFrame([Acc_Train[vals]*100 for vals in Acc_Train],columns=['Accuracy Train'],index=[vals for vals in Acc_Train])
Std_train = pd.DataFrame([Std_Train[vals]*100 for vals in Std_Train],columns=['S. Deviation Train'],index=[vals for vals in Std_Train])
Accuracy_pred = pd.DataFrame([Acc_Test[vals]*100 for vals in Acc_Test],columns=['Accuracy Test'],index=[vals for vals in Acc_Test])
Std_test = pd.DataFrame([Std_Test[vals]*100 for vals in Std_Test],columns=['S. Deviation Test'], index=[vals for vals in Std_Test])
ROC_Area = pd.DataFrame([ROC[vals] for vals in ROC],columns=['ROC(area)'],index=[vals for vals in ROC])
AUC_Area = pd.DataFrame([AUC[vals] for vals in AUC],columns=['AUC(area)'],index=[vals for vals in AUC])
Gm = pd.DataFrame([Gmean[vals] for vals in Gmean],columns=['Gmean'],index=[vals for vals in Gmean])
Prec = pd.DataFrame([Precision[vals] for vals in Precision],columns=['Precision'],index=[vals for vals in Precision])
Rec = pd.DataFrame([Recall[vals] for vals in Recall],columns=['Recall'],index=[vals for vals in Recall])
F1 =  pd.DataFrame([F1_score[vals] for vals in F1_score],columns=['F1_score'],index=[vals for vals in F1_score])

table = pd.concat([Accuracy_train,Std_train,Accuracy_pred,Std_test,ROC_Area,AUC_Area,Gm,Prec,Rec,F1], axis=1)
table.loc['MEAN VALUE'] = table.mean()
table

In [ ]:
k_range=list(range(1,60))
data_copy=pd.Series()
k_scores = []
#x=[0,1,2,3,4]
for i in k_range:
    model=KNeighborsClassifier(n_neighbors=i) 
    scores = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')
    k_scores.append(scores.mean())
    model.fit(X_train,y_train)
    pred_test=model.predict(X_test)
    data_copy=data_copy.append(pd.Series(metrics.accuracy_score(pred_test,y_test)))

#plt.plot(k_range, k_scores)
plt.plot(k_range, data_copy)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated Accuracy')    
#plt.xticks(X)
plt.show()
print('Accuracies for different values of n are:\n', data_copy.values)

In [ ]:
'''svc = SVC(probability=True, random_state=random_state)
# Set the parameters by cross-validation
svc_pg = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4],'C': [1, 10, 100, 1000]},
          {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    svc.fit(X_train, y_train)
    y_eval = svc.predict(X_test)
    acc = sum(y_eval == y_test) / float(len(y_test))
    print("Accuracy of SVC: %.2f%%" % (100*acc))
    
    print("Tuning hyper-parameters for %s\n" % score)
    svc_gscv = GridSearchCV(svc, svc_pg, cv=kfold, scoring='%s_macro' % score)
    svc_gscv.fit(X_train, y_train)
    print("Best parameters set found on training set:")
    print(svc_gscv.best_params_,"\n")
    print("Grid scores on training set:")
    means = svc_gscv.cv_results_['mean_test_score']
    stds = svc_gscv.cv_results_['std_test_score']
    
    for mean, std, params in zip(means, stds, svc_gscv.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print("Detailed classification report:")
    print("The model is trained on the full training set.")
    print("The scores are computed on the full test set.")
    y_true, y_pred = y_test, svc_gscv.predict(X_test)
    print(classification_report(y_true, y_pred))
    
    svc_est = svc_gscv.best_estimator_
    svc_score = svc_gscv.best_score_
    print("Best estimator for parameter C: %f\n" % (svc_est.C))
    print("Best score: %0.2f%%\n" % (100*svc_score))

    #print(clf)'''

In [ ]:
gnb = GaussianNB()

In [ ]:
mlp = MLPClassifier(momentum=0.15,solver='sgd',learning_rate_init=1.0, early_stopping=True, shuffle=True,random_state=random_state)

mlp_pg={
'learning_rate': ["constant", "invscaling", "adaptive"],
'hidden_layer_sizes': [(10,10),(100,),(100,10)],
#'hidden_layer_sizes': [x for x in itertools.product((10,20,30,40,50,100),repeat=3)],
#'tol': [1e-2, 1e-3, 1e-4],
#'epsilon': [1e-3, 1e-7, 1e-8],
'alpha': [1e-2, 1e-3, 1e-4],
#'activation': ["logistic", "relu", "Tanh"]
}

mlp_gscv = GridSearchCV(mlp,param_grid=mlp_pg, cv=kfold, scoring="accuracy", n_jobs= -1, verbose = 1)
mlp_gscv.fit(X_train, y_train)

mlp_est = mlp_gscv.best_estimator_
mlp_score = mlp_gscv.best_score_
print("Best estimator:", mlp_est, "\nBest Score:", mlp_score)

In [ ]:
lr = LogisticRegression(
    C=0.1,
    penalty='l2',
    dual=True,
    tol=0.0001, 
    fit_intercept=True,
    intercept_scaling=1.0, 
    class_weight=None,
    random_state=43)

lr_pg = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

lr_gscv = GridSearchCV(lr,param_grid=lr_pg, cv=kfold, scoring="accuracy", n_jobs= -1, verbose = 1)
lr_gscv.fit(X_train, y_train)

lr_est = lr_gscv.best_estimator_
lr_score = lr_gscv.best_score_
print("Best estimator:", lr_est, "\nBest Score:", lr_score)

In [ ]:
predictor1=LogisticRegression(C=0.001, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1.0, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=43, solver='warn',
          tol=0.0001, verbose=0, warm_start=False).fit(X_train, y_train)
prediction=predictor1.predict(X_test)
test_healthy = pd.Series(prediction, name="Healthy")
predictor1.score(X_test,y_test)

In [ ]:
submission = pd.DataFrame({'Healthy': prediction })
submission.to_csv("Results_df_NSR.csv", index=True)
print(submission)

In [ ]:
xgbc = xgb.XGBClassifier(
    #learning_rate = 0.02,
     n_estimators= 2000,
     max_depth= 4,
     min_child_weight= 2,
     gamma=0.9,                        
     subsample=0.8,
     colsample_bytree=0.8,
     objective= 'binary:logistic',
     nthread= -1,
     scale_pos_weight=1)

xgbc.fit(X_train, y_train)


X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, y_train, test_size=0.3, random_state=random_state)
train2 = xgb.DMatrix(X_train2, y_train2)
test2 = xgb.DMatrix(X_test2, y_test2)
watchlist = [(test2, 'test')]
xgb_pg = {
    'max_depth': 3,
    'booster': 'gbtree',
    'objective': 'reg:linear',#'binary:logistic'
    'subsample': 0.8,
    'colsample_bytree': 0.85,
    'eta': 0.05,
    'max_depth': 7,
    'seed': 43,
    'silent': 0,
    'eval_metric': 'rmse' #root mean square estimate
}

bst = xgb.train(xgb_pg, train2, 500, watchlist, early_stopping_rounds=50)
#bst.save_model("model_new")
#bst = xgb.Booster(xgb_pg)
#bst.load_model("model_new")

# make prediction
#pred = bst.predict(xgb.DMatrix(X_test))
prediction = xgbc.predict(X_test)
pred = bst.predict(test2,ntree_limit=bst.best_ntree_limit)
xgb.to_graphviz(bst, num_trees=2)

In [ ]:
submission = pd.DataFrame({'Healthy': prediction })
submission.to_csv("Results_df_NSR.csv", index=True)
print(submission)

In [ ]:
class Ensemble(object):
    def __init__(self, n_folds, stacker, base_models):
        self.n_folds = n_folds
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(KFold(len(y), n_folds=self.n_folds, shuffle=True, random_state=2016))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))

        for i, clf in enumerate(self.base_models):
            S_test_i = np.zeros((T.shape[0], len(folds)))

            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                # y_holdout = y[test_idx]
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_holdout)[:]
                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict(T)[:]

            S_test[:, i] = S_test_i.mean(1)

        self.stacker.fit(S_train, y)
        y_pred = self.stacker.predict(S_test)[:]
        return y_pred

In [ ]:
gnb_mlp=VotingClassifier(estimators=[('Guassian Naive Bayes', gnb),('Multilayer Perceptron',mlp)], voting='soft', weights=[2,1]).fit(X_train,y_train)
print('The accuracy for Guassian Naive Bayes and Multilayer Perceptron:',gnb_mlp.score(X_test,y_test))

In [ ]:
gnb_lr=VotingClassifier(estimators=[('Guassian Naive Bayes', gnb),('Logistic Regression', lr)], voting='soft', weights=[2,1]).fit(X_train,y_train)
print('The accuracy for Guassian Naive Bayes and Logistic Regression:',gnb_lr.score(X_test,y_test))

In [ ]:
mlp_lr=VotingClassifier(estimators=[('Multilayer Perceptron',mlp), ('Logistic Regression', lr)], voting='soft', weights=[2,1]).fit(X_train,y_train)
print('The accuracy for Multilayer Perceptron and Logistic Regression:',mlp_lr.score(X_test,y_test))

In [ ]:
gnb_mlp_lr=VotingClassifier(estimators=[('Guassian Naive Bayes', gnb),('Multilayer Perceptron',mlp), ('Logistic Regression', lr)], voting='soft', weights=[3,2,1]).fit(X_train,y_train)
print('The ensembled model with Guassian Naive Bayes, Multilayer Perceptron, and Logistic Regression:',gnb_mlp_lr.score(X_test,y_test))

In [ ]:
# Generate a simple plot of the test and training learning curve
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)): 
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",label="Cross-validation score")
    plt.legend(loc="best")
    return plt

#gnb_plot = plot_learning_curve(gnb_est,"GNB learning curves",X_train,Y_train,cv=kfold)
mlp_plot = plot_learning_curve(mlp_est,"MLP learning curves",X_train,y_train,cv=kfold)
lr_plot = plot_learning_curve(lr_est,"LR learning curves",X_train,y_train,cv=kfold)
svc_plot = plot_learning_curve(svc_est,"SVC learning curves",X_train,y_train,cv=kfold)
#xgb_plot = plot_learning_curve(xgb_gscv.best_estimator_,"XBG learning curves",X_train,Y_train,cv=kfold)


In [ ]:
#test_survived_gnb = pd.Series(gnb_est.predict(test), name="GNBC")
test_Survived_mlp = pd.Series(mlp_est.predict(X_test), name="MLP")
test_Survived_lr = pd.Series(lr_est.predict(X_test), name="LR")
test_Survived_svc = pd.Series(svc_est.predict(X_test), name="SVC")

# Concatenate all classifier results
ensemble_results = pd.concat([test_Survived_mlp,test_Survived_lr,test_Survived_svc],axis=1)
g= sns.heatmap(ensemble_results.corr(),annot=True)

In [ ]:
finalVC = VotingClassifier(estimators=[('Logistic Regression', lr),('SVM',svc)], voting='soft', weights=[2,1], n_jobs = -1).fit(X_train,y_train)
print('The ensembled model with all classfiers: ',finalVC.score(X_test,y_test))

# Generate Submission File 
test_healthy = pd.Series(finalVC.predict(X_test), name="Healthy")
#results = pd.concat([X_test['index'],test_healthy],axis=1)
#results.to_csv("Results_series_NSR.csv", index=False)
#submission1 = pd.DataFrame({'Healthy Probabilty': pred })
#submission1.to_csv("Results_df1_NSR.csv", index=True)
#print(submission1)


In [ ]:
submission = pd.DataFrame({'Healthy': prediction })
submission.to_csv("Results_df_NSR.csv", index=True)
print(submission)